# GitHub Search with Existing Browser Connection

This example demonstrates using webtask's autonomous agent to search GitHub and extract results, all while connected to your existing Chrome browser.

## Setup

Before running this notebook, launch Chrome with debugging enabled:

```bash
# On Mac/Windows:
chrome --remote-debugging-port=9222 --user-data-dir=~/Desktop/chrome-automation

# On Linux:
google-chrome --remote-debugging-port=9222 --user-data-dir=~/Desktop/chrome-automation
```

## Requirements

- Chrome/Chromium launched with `--remote-debugging-port=9222`
- `GOOGLE_API_KEY` environment variable set

In [ ]:
# Imports
import asyncio
import os
from webtask import Webtask
from webtask.integrations.llm import GeminiLLM
from webtask.integrations.browser.playwright import PlaywrightBrowser

## Connect to Existing Browser

Connect to your Chrome browser that's already running with debugging enabled.

In [ ]:
# Connect to existing Chrome browser
print("📡 Connecting to existing Chrome browser...")
browser = await PlaywrightBrowser.connect("http://localhost:9222")
print("✅ Connected!")

## Create Agent

Create a webtask agent that will control your browser autonomously.

In [ ]:
# Create LLM
api_key = os.getenv("GOOGLE_API_KEY")
llm = GeminiLLM(model="gemini-2.5-flash", api_key=api_key)

# Create Webtask and agent using the connected browser
wt = Webtask()
agent = await wt.create_agent_with_browser(
    llm=llm,
    browser=browser,
)

print("🤖 Agent ready!")

## Execute Task

Now let the agent autonomously search GitHub for 'playwright' and extract the top result.

Watch your Chrome window - you'll see the agent navigate, search, and read results automatically!

In [ ]:
# Execute autonomous task
print("🔍 Executing task: Search GitHub for 'playwright' and get the top result...\n")

result = await agent.do(
    task="First Go to github and finds out the url for webtask and then go to my linkedin page and then send the following message to my connection Vivi Wei: Hi Luke, this is hello from webtask, check out the repo at XXX",
    mode="dom"
)

print("\n" + "="*60)
print("📊 Task Result:")
print("="*60)
print(f"Status: {result['status']}")
print(f"Feedback: {result['feedback']}")
if result.get('output'):
    print(f"Output: {result['output']}")

## Cleanup

Disconnect from the browser (but keep it running).

In [ ]:
# Cleanup: Close the Playwright connection (but not the customer's browser!)
await browser.close()
print("👋 Disconnected. Your browser stays open.")